In [54]:
from pyspark.sql import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import spark_partition_id

In [55]:
conf = SparkConf() \
      .setMaster("local[3]") \
      .setAppName("SparkSQLTable")

spark = SparkSession \
      .builder \
      .config(conf=conf) \
      .enableHiveSupport() \
      .getOrCreate()

In [56]:
flight_parquet_df = spark.read \
    .format("parquet") \
    .load("./../data/flight*.parquet") 

flight_parquet_df.show()

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

In [57]:
# Create database

spark.sql("CREATE DATABASE IF NOT EXISTS airline_db")
spark.catalog.setCurrentDatabase("airline_db")

In [58]:
flight_parquet_df.write \
      .format("csv") \
      .mode("overwrite") \
      .bucketBy(5, "OP_CARRIER", "ORIGIN") \
      .sortBy("OP_CARRIER", "ORIGIN") \
      .saveAsTable("flight_data_tbl")

23/08/25 23:47:25 WARN HadoopFSUtils: The directory file:/Users/johnbelga/workspace/python/spark/lab/lessons/spark-warehouse/airline_db.db/flight_data_tbl was not found. Was it deleted very recently?
23/08/25 23:47:25 WARN FileUtils: File file:/Users/johnbelga/workspace/python/spark/lab/lessons/spark-warehouse/airline_db.db/flight_data_tbl does not exist; Force to delete it.
23/08/25 23:47:25 ERROR FileUtils: Failed to delete file:/Users/johnbelga/workspace/python/spark/lab/lessons/spark-warehouse/airline_db.db/flight_data_tbl
23/08/25 23:47:28 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`airline_db`.`flight_data_tbl` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
23/08/25 23:47:28 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/08/25 23:47:28 WARN HiveCon

In [59]:
spark.stop()